# Kafka Data Consumer

## I/ Objectifs du notebook

## II/ Mécanisme de collecte en streaming des données


In [1]:
kafka_server ='localhost:9092'
#kafka_server ='islin-hdpnod1:6667'
#kafka_server ='islin-hdplnod06:6667'

In [ ]:
import kairosdb_utils
global kairosdb_server 
kairosdb_server = "http://localhost:9080"

In [ ]:
import influxdb_utils
db_host= 'localhost'
port = '8086'
db_name='TimeSeriesBench'
influxdb_client = influxdb_utils.influxdb_connect(db_host, port)

In [2]:
def consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,insert_data,convert_to_dbdata):
    print("CONNECTION KAFKA : ",kafka_server)
    print("TOPIC : ",topic)
    
    scheme = get_scheme(topic)
    consumer = KafkaConsumer(bootstrap_servers=kafka_server)

    tp = TopicPartition(topic, 0)
    consumer.assign([tp])
    consumer.seek_to_end(tp)
    lastOffset = consumer.position(tp)
    print("RANGE:",tp,lastOffset)
    consumer.seek_to_beginning(tp)
    # end
    try :
        while True:
            list_data = []
            for message in consumer:
                msg = message.value.decode()
                list_data.append(msg)
                if len(list_data) == bucket_size:
                    print(str(len(list_data)) + " data fetched from topic : " + topic)
                    new_data = convert_to_dbdata(list_data)
                    insert_data(db_name, coll_name, new_data)
                    list_data.clear()
                    if len(list_data) == lastOffset:
                        return

    finally:
        # Close down consumer to commit final offsets.
        consumer.close()

## III/ Mécanismes de validation et corrections des données

In [3]:
def get_scheme(topic):
    if "SmartGrid" in topic:
        scheme = {"timestamp":{'type': 'string','required': True,'empty': False},
                  "tagname":{'type': 'string','required': True,'empty': False},
                  "value":{'type': 'string','required': True,'empty': False},
                  "quality":{'type': 'string','required': True,'empty': False}}
        return scheme
    if "WindProp" in topic :
        scheme = {  "Heure":{'type': 'string','required': True,'empty': False},
                    "Temps écoulé":{'type': 'string','required': True,'empty': False},
                    '4069 state':{'type': 'string','required': True,'empty': False},
                    'Battery voltage':{'type': 'string','required': True,'empty': False},
                    'Flow SP':{'type': 'string','required': True,'empty': False},
                    'Unit Code':{'type': 'string','required': True,'empty': False},
                    'Flow M':{'type': 'string','required': True,'empty': False},
                    'Pressure1':{'type': 'string','required': True,'empty': False},
                    'Temperature1':{'type': 'string','required': True,'empty': False},
                    "Latitude":{'type': 'string','required': True,'empty': False},
                    "Longitude":{'type': 'string','required': True,'empty': False},
                    "Altitude":{'type': 'string','required': True,'empty': False},
                    "Head_Rel_True North":{'type': 'string','required': True,'empty': False},
                    "Pressure2":{'type': 'string','required': True,'empty': False},
                    "Temperature2":{'type': 'string','required': True,'empty': False},
                    "Humidity":{'type': 'string','required': True,'empty': False},
                    "MDA Wnd Dir":{'type': 'string','required': True,'empty': False},
                    "MDA Wnd Speed":{'type': 'string','required': True,'empty': False},
                    "MWD Wind Dir":{'type': 'string','required': True,'empty': False},
                    "MWD Wind Speed":{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 1':{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 2':{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 3':{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 4':{'type': 'string','required': True,'empty': False},
                    'SPA 1':{'type': 'string','required': True,'empty': False},
                    'SPA 2':{'type': 'string','required': True,'empty': False},
                    'Cellule Photo':{'type': 'string','required': True,'empty': False},
                    'Temperature3':{'type': 'string','required': True,'empty': False},
                    'Pressure3':{'type': 'string','required': True,'empty': False},
                    'Flow MassFlow 1':{'type': 'string','required': True,'empty': False},
                    'NOTUSED Flow MassFlow 2':{'type': 'string','required': True,'empty': False},
                    'Flow':{'type': 'string','required': True,'empty': False},
                    'Humidity2':{'type': 'string','required': True,'empty': False},
                    'Test':{'type': 'string','required': True,'empty': False},
                    'Details':{'type': 'string','required': True,'empty': False},
                    'SPA 3':{'type': 'string','required': True,'empty': False},
                    'SPA 4':{'type': 'string','required': True,'empty': False},
                    "CavityPressure":{'type': 'string','required': True,'empty': False},
                    "CavityTemp":{'type': 'string','required': True,'empty': False},
                    "CH4":{'type': 'string','required': True,'empty': False},
                    "CH4_dry":{'type': 'string','required': True,'empty': False},
                    "C2H6":{'type': 'string','required': True,'empty': False},
                    "C2H6_dry":{'type': 'string','required': True,'empty': False},
                    "13CH4":{'type': 'string','required': True,'empty': False},
                    "H2O":{'type': 'string','required': True,'empty': False},
                    "CO2":{'type': 'string','required': True,'empty': False},
                    "C2C1Ratio":{'type': 'string','required': True,'empty': False},
                    "Delta_iCH4_Raw":{'type': 'string','required': True,'empty': False},
                    "HP_Delta_iCH4_30s":{'type': 'string','required': True,'empty': False},
                    "HP_Delta_iCH4_2min":{'type': 'string','required': True,'empty': False},
                    "HP_Delta_iCH4_5min":{'type': 'string','required': True,'empty': False} }
        return scheme

In [4]:
def clean_data(data,scheme):
    v = Validator(scheme)
    for index,item in enumerate(data,start=0):
        res = v.validate(item)
        if (res == False):
            print("corrupt data in line :",index,", error : ",v.errors)
            del data[index]

## IV/ Mécanismes de mise en base des données

In [5]:
import numpy as np
import sys
sys.path.append("../TimeSeriesTools")

In [6]:
import mongodb_utils
db_host= 'localhost'
port = '27018'
db_name='TimeSeriesBench'
mongodb_client = mongodb_utils.mongodb_connect(db_host, port)

Trying to connect to MongoDB server: localhost on port: 27018


### A/ Mise en base du schéma des données

In [ ]:
def str_to_unix(date):
    dt = datetime.strptime(date, '%d/%m/%Y %H:%M:%S')
    epoch = datetime.utcfromtimestamp(0)
    return int((dt - epoch).total_seconds()) * 1000

In [ ]:
def type_convert(data):
    data = data.replace(',', '.')
    time = str_to_unix(data.split(";", 1)[0].split(":", 1)[1])
    data_treated = [time, data.split(";", 1)[1]]
    return data_treated

In [ ]:
def mongodb_convert_to_dbdata(list_data,scheme):
    col= [k for k in scheme.keys()]
    new_dict=[]
    
    t0 = time.process_time()
    for item in list_data:
        info=item.split(":",1)[1].split(";")
        dict_item ={col[i]:info[i] for i in range(len(col))}
        new_dict.append(dict_item)
    clean_data(new_dict)
    print('.. %f seconds for validate' % (t1 - t0))
    
    return new_dict

In [ ]:
def db_convert_to_dbdata(list_data,scheme):
    col = [k for k in scheme.keys()]
    v = Validator(scheme)
    db_data = []
    t0 = time.process_time()
    for index,item in enumerate(list_msg):
        info = item.split(":", 1)[1].split(";")
        dict_item = {col[i]: info[i] for i in range(len(col))}
        res = v.validate(dict_item)
        if res == True:
            new_data = type_convert(item)
            db_data.append(new_data)
        else:
            print("corrupt data in line :",index,", error : ",v.errors)
    t1 = time.process_time()
    print('.. %f seconds for validate' % (t1 - t0))
    return db_data

In [7]:
def mongodb_insert_data(db_name, coll_name, list_data):
    mongodb_utils.insert_many_docs(mongodb_client,db_name, coll_name,list_data)    
    return len(list_data)

In [ ]:
def kairosdb_insert_data(db_name, coll_name, list_data):
    kairosdb_utils.insert_many_docs(kairosdb_server,db_name, coll_name,list_data)    
    return len(list_data)

In [ ]:
def influxdb_insert_data(db_name, coll_name, list_data):
    influxdb_utils.insert_many_docs(influxdb_client,db_name, coll_name,list_data)    
    return len(list_data)

#### 1) Cas test SmartGrid

In [9]:
topic = 'SmartGridCryolite20190101OneMonthTopic'
scheme = get_scheme(topic)

In [10]:
db=mongodb_client[db_name]
db.list_collection_names()

['WindPropLacqOneDayP25',
 'schemes',
 'WindPropLacqOneDay',
 'WindPropLacqOneDayV2']

In [21]:
scheme_collection = db['schemes']

In [22]:
scheme_id = scheme_collection.insert_one(scheme)

In [23]:
scheme_id

In [ ]:
coll_name='SmartGridCryolite20190101OneMonth'

In [ ]:
%%time
bucket_size = 1000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,mongodb_insert_data,mongodb_convert_to_dbdata)

In [ ]:
%%time
bucket_size = 1000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,kairosdb_insert_data,db_convert_to_dbdata)

In [ ]:
%%time
bucket_size = 1000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,influxdb_insert_data,db_convert_to_dbdata)

### 2) Cas test WindProp

In [4]:
topic = 'WindPropDataLacqOneDayP25Topic'

In [ ]:
scheme = get_scheme(topic)

In [11]:
db = mongodb_client[db_name]
scheme_collection = db['schemes']

In [13]:
scheme_id = scheme_collection.insert_one(scheme)

In [14]:
scheme_id

In [ ]:
coll_name='WindPropLacqOneDayP25'

In [ ]:
%%time
bucket_size = 1000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,mongodb_insert_data,mongodb_convert_to_dbdata)

In [ ]:
%%time
bucket_size = 1000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,kairosdb_insert_data,db_convert_to_dbdata)

In [ ]:
%%time
bucket_size = 1000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,influxdb_insert_data,db_convert_to_dbdata)

### 3) Cas test Pollution